# 1.2-agifford-AnalyzeSingleDataFile
This notebook performs exploratory data analysis on an example datafile.

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

parq_file = "../../data/interim/raw/fileID1_subjID3_dataID0.parquet"
df = pd.read_parquet(parq_file, engine="fastparquet")

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
def _make_single_annot_frame(df, shift):
    annot_df = df[df.label != df.label.shift(shift)]
    annot_df = annot_df.dropna(subset="label")
    return annot_df

def make_annot_dataframe(df, t_start=None, t_end=None):
    t_start = t_start or df.time.min()
    t_end = t_end or df.time.max()
    
    df = df[(df.time >= t_start) & (df.time <= t_end)].copy()
    
    (act_starts_df, act_ends_df) = (
        _make_single_annot_frame(df, shift) for shift in [1, -1]
    )
    return act_starts_df, act_ends_df

In [3]:
# not sure I'll need the starts & ends df, but probably should remove the rows with no 
# activity labels
activity_starts_df, activity_ends_df = make_annot_dataframe(df)
df_dropna =  df.dropna(subset="label")

In [4]:
df.shape, df_dropna.shape

((127959, 12), (127927, 12))

In [5]:
df.columns

Index(['time', 'file_id', 'subject_id', 'data_id', 'accel_x', 'accel_y',
       'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label', 'label_group'],
      dtype='object')

In [10]:
df_dropna.label.unique()

array(['<Initial Activity>', 'Tap Left Device', 'Tap Right Device',
       'Non-Exercise', 'Jumping Jacks', 'Plank', 'Squat Jump', 'Sit-ups',
       'Dumbbell Deadlift Row', 'Side Plank Right side',
       'Side Plank Left side', 'Bicep Curl', 'Pushups', 'Walking lunge',
       'Power Boat pose', 'Squat', 'Walk'], dtype=object)

In [8]:
df_dropna.label_group.unique()

array(['Junk', 'Non-exercise', 'Legacy activity', 'Static exercise',
       'Repetitive exercise', 'Repetitive non-arm exercise', 'Walk'],
      dtype=object)

For the sake of getting through this project end-to-end, I will not spend too much time on building a very sophisticated model. As such, I will use the column `label_group` as my desired prediction column.

Process for each `label_group`:
1. Compute FFT with a hamming window for each instance of the `label_group`
2. Average the FFTs across instances
3. Identify the major frequencies of the group by setting some arbitrary threshold to identify peaks.
4. I will use those frequencies to generate `sin` and `cos` features as inputs to a basic model to predict `label_group`.
5. Repeat steps 1-4 for each variable x direction combination (e.g., "accel_x", "accel_y", etc.)

First, let's template out the process of analyzing a single instance of a single `label_group`.

In [13]:
pd.concat([activity_starts_df.head(1), activity_ends_df.head(1)], ignore_index=True)

,time,file_id,subject_id,data_id,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,label,label_group
0,0.000000,1,3,0,-1.053396,-0.240851,0.007644,-6.320330,-38.337973,-16.067351,<Initial Activity>,Junk
1,102.999759,1,3,0,0.069121,-0.901886,0.737462,0.569303,60.823368,98.224509,<Initial Activity>,Junk
